In [3]:
import onnxruntime as ort
import numpy as np
from PIL import Image

def upscale_image_with_onnx(model_path, input_image_path, output_image_path):
    # Load the ONNX model
    session = ort.InferenceSession(model_path)

    # Load and preprocess the input image
    image = Image.open(input_image_path).convert("RGB")
    input_size = session.get_inputs()[0].shape[2:]  # Get input size from the model
    input_array = np.array(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
    input_array = np.transpose(input_array, (2, 0, 1))  # HWC to CHW
    input_array = np.expand_dims(input_array, axis=0)  # Add batch dimension

    # Run the model
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    input_array = input_array.astype(np.float16)  # Convert input to float16
    output_array = session.run([output_name], {input_name: input_array})[0]

    # Postprocess the output
    output_array = np.squeeze(output_array)  # Remove batch dimension
    output_array = np.transpose(output_array, (1, 2, 0))  # CHW to HWC
    output_array = np.clip(output_array * 255.0, 0, 255).astype(np.uint8)  # Denormalize to [0, 255]
    output_image = Image.fromarray(output_array)

    # Save the upscaled image
    output_image.save(output_image_path)
    print(f"Upscaled image saved to {output_image_path}")

# Example usage
model_path = r"onnxModel/Restore_UL.onnx"
input_image_path = r"d:\3dModel\Hueforge_image\kyockcho\PROCESSED\Miyabi.png"
output_image_path = r"R:\out.png"
upscale_image_with_onnx(model_path, input_image_path, output_image_path)

Upscaled image saved to R:\out.png
